In [3]:
import autogen
import langchain
import llama_index
import openai
import requests
import xml.etree.ElementTree as ET
from scholarly import scholarly

print("✅ All libraries imported successfully!")



✅ All libraries imported successfully!


In [ ]:
import os
from getpass import getpass

# Get API Key securely
tokenGROQ = getpass('Enter GROQ_API_KEY here: ')
os.environ["GROQ_API_KEY"] = tokenGROQ

In [ ]:
from autogen_ext.models.openai import OpenAIChatCompletionClient

# Step 1: Create the model client
model_client = OpenAIChatCompletionClient(
    model="llama3-70b-8192",
    base_url="https://api.groq.com/openai/v1",
    api_key=os.environ["GROQ_API_KEY"],
    model_info={
        "vision": False,
        "function_calling": True,
        "json_output": True,
        "family": "llama3",
    },
)

# Step 2: Verify the client is set up correctly
print(model_client)


In [7]:
def fetch_google_scholar_papers(query):
        """
            Fetches top 6 research papers from Google Scholar.
            Returns:
                list: A list of dictionaries containing paper details (title, summary, link)
        """
        papers = []
        search_results = scholarly.search_pubs(query)

        for i, paper in enumerate(search_results):
            if i >= 6:
                break
            papers.append({
                "title": paper["bib"]["title"],
                "summary": paper["bib"].get("abstract", "No summary available"),
                "link": paper.get("pub_url", "No link available")
            })
        return papers



In [8]:
res=fetch_google_scholar_papers("Multi-Agent LLM Systems")
for i, paper in enumerate(res):
    print(f"\n📄 Paper {i+1}: {paper['title']}")
    print(f"📝 Summary: {paper['summary']}")
    print(f"🔗 Link: {paper['link']}")



📄 Paper 1: A survey on LLM-based multi-agent systems: workflow, infrastructure, and challenges
📝 Summary: For system  LLM-based multi-agent systems in two main areas: problem-solving and world  simulation. Finally, we address several challenges faced by LLM-based multi-agent systems
🔗 Link: https://link.springer.com/article/10.1007/s44336-024-00009-2

📄 Paper 2: LLM multi-agent systems: Challenges and open problems
📝 Summary: multi-agent systems. We also explore the potential application of multi-agent systems in  blockchain systems to  their future development and application in real-world distributed systems.
🔗 Link: https://arxiv.org/abs/2402.03578

📄 Paper 3: Multi-agent collaboration: Harnessing the power of intelligent llm agents
📝 Summary: In fact, our proposed multi-agent system also aims to make strides toward achieving a higher  level of artificial general intelligence (AGI). The development and deployment of advanced
🔗 Link: https://arxiv.org/abs/2306.03314

📄 Paper 4: Llm-

In [ ]:
from autogen_agentchat.agents import AssistantAgent

# Summarizer Agent
summarizer_agent = AssistantAgent(
    name="SummarizerAgent",
    model_client=model_client,
    description="An agent that summarizes research paper content.",
    system_message="You are an AI assistant that reads and summarizes research papers, extracting key points concisely."
)

print("✅ Summarizer Agent Initialized!")

In [ ]:
insight_extractor_agent = AssistantAgent(
    name="InsightExtractorAgent",
    model_client=model_client,
    description="An agent that extracts key insights and findings from research papers.",
    system_message="You analyze research paper summaries and extract key insights, advantages, disadvantages, and conclusions."
)

print("✅ Insight Extractor Agent Initialized!")

In [ ]:
# Review Composer Agent
review_composer_agent = AssistantAgent(
    name="ReviewComposerAgent",
    model_client=model_client,
    description="An agent that composes a structured literature review based on extracted insights.",
    system_message="You compile extracted research insights into a well-structured, 500-word literature review, maintaining clarity and coherence."
)

print("✅ Review Composer Agent Initialized!")


In [ ]:
from autogen_agentchat.agents import UserProxyAgent

# User Proxy Agent for human oversight
user_proxy_agent = UserProxyAgent(
    name="HumanReviewerAgent",
    human_input_mode="ALWAYS",
    description="A user agent that reviews AI-generated literature reviews and refines them.",
    system_message="You assist in reviewing, modifying, and refining AI-generated literature reviews for better accuracy and readability."
)

print("✅ Human Reviewer Agent Initialized!")
